provide a Python implementation of a Hybrid ARIMA-ML model for CPI forecasting. This will include:

ARIMA for Base Forecasting
Residual Computation
LSTM Model to Predict Residuals
Final CPI Forecast (ARIMA + LSTM Prediction)

In [ ]:
pip install numpy pandas matplotlib statsmodels tensorflow scikit-learn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Load CPI data (Example: Monthly CPI data)
df = pd.read_csv("cpi_data.csv", parse_dates=['Date'], index_col='Date')
df = df.asfreq('M')  # Ensure data is monthly
df.dropna(inplace=True)  # Remove missing values

# Visualize the CPI trend
plt.figure(figsize=(10,5))
plt.plot(df, label="CPI")
plt.title("CPI Time Series Data")
plt.legend()
plt.show()


In [ ]:
# Fit ARIMA Model (p=2, d=1, q=2 is an example, optimize for best results)
arima_model = ARIMA(df, order=(2,1,2))
arima_result = arima_model.fit()

# Generate ARIMA Forecast
df['ARIMA_Forecast'] = arima_result.fittedvalues

# Compute Residuals (Errors)
df['Residuals'] = df.iloc[:, 0] - df['ARIMA_Forecast']

# Plot Residuals
plt.figure(figsize=(10,5))
plt.plot(df['Residuals'], label="Residuals")
plt.title("Residuals (CPI - ARIMA Prediction)")
plt.legend()
plt.show()


In [ ]:
# Normalize Residuals
scaler = MinMaxScaler()
df['Residuals_Scaled'] = scaler.fit_transform(df[['Residuals']])

# Create sequences for LSTM
def create_sequences(data, seq_length=10):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# Define sequence length
SEQ_LENGTH = 10  # Lookback window

# Prepare training data
X, y = create_sequences(df['Residuals_Scaled'].values, SEQ_LENGTH)

# Reshape for LSTM [Samples, TimeSteps, Features]
X = X.reshape(X.shape[0], X.shape[1], 1)


In [ ]:
# Define LSTM Model
model = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(SEQ_LENGTH, 1)),
    LSTM(50, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=50, batch_size=16, verbose=1)

# Predict Residuals
predicted_residuals = model.predict(X)
predicted_residuals = scaler.inverse_transform(predicted_residuals)

# Align Predictions with DataFrame
df['Predicted_Residuals'] = np.nan
df.iloc[SEQ_LENGTH:, df.columns.get_loc('Predicted_Residuals')] = predicted_residuals[:, 0]


In [ ]:
df['Final_Forecast'] = df['ARIMA_Forecast'] + df['Predicted_Residuals']

# Plot Results
plt.figure(figsize=(12,6))
plt.plot(df.iloc[:, 0], label="Actual CPI", color='black')
plt.plot(df['ARIMA_Forecast'], label="ARIMA Forecast", linestyle="dashed", color='blue')
plt.plot(df['Final_Forecast'], label="Hybrid ARIMA-LSTM", linestyle="dashed", color='red')
plt.title("Hybrid ARIMA-LSTM CPI Forecasting")
plt.legend()
plt.show()


In [ ]:
✅ ARIMA captures linear trends & seasonality
✅ LSTM models nonlinear residuals (shocks, external factors)
✅ Final model improves accuracy over standalone ARIMA or LSTM

Would you like to extend this to ARIMA-XGBoost or add external economic indicators (e.g., GDP, interest rates)? 🚀